In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 100)
# # ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# # Processes = []
# # for i in range(1,16):
# #     Processes.append(i*i)
# # for j in range(9):
# #     Processes.append(2**j)
# # Processes = list(set(Processes))
# # Processes.sort()
# # index = classes
# # EmptyDF = pd.DataFrame(index=index, columns=Processes)
# # for benchmark in benchmarks:
# #     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

# ##### 各ベンチマークの実行時間を記録する処理
# # # Benchmarks = []
# # # for benchmark in benchmarks:
# # #     Benchmarks.append(benchmark.upper())
# # # print(Benchmarks)

# # Processes = [8, 16, 32, 64, 128, 256]
# # for process in Processes:
# #     TimeDataList = return_TimeDataList(process)
# #     for DictData in TimeDataList:
# #         benchmark = DictData.benchmark
# #         process = DictData.process
# #         BenchmarkClass = DictData.BenchmarkClass
# #         time = DictData.time
# #         FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# # # print(TimeDataList[0].return_AllData())




# # for benchmark in ['bt', 'mg', 'sp']:
# FixClass = "B"
# for benchmark in ['mg']:
#     print(f"benchmark={benchmark}")
#     TargetDF = return_FixedClassModelDF(benchmark=benchmark, FixClass=FixClass)
#     BestModelsInTargetDF = BestModelsInDF(TargetDF)
#     print(BestModelsInTargetDF)
#     TargetDF = return_ErrorRateFixedClassDFwithAverage(benchmark = benchmark , FixClass=FixClass)
#     print(TargetDF)

# # for benchmark in ['bt', 'mg', 'sp']:
# for benchmark in ['mg']:
#     ProcessExcludeBTSP = [1,2,4,8,16,32,64,128,256]
#     ProcessIncludeBTSP = [1, 4, 16, 64, 256]

#     TestRates = [0, 0.2, 0.4, 0.6, 0.8]
#     if(benchmark == "bt" or benchmark == "sp"):
#         processes=ProcessIncludeBTSP
#     else:
#         processes=ProcessExcludeBTSP
#     RawDataList = return_fixed_class(BenchMark=benchmark, Processes=processes,FixedBenchMarkClass=FixClass)
#     RawDataDF = pd.concat(RawDataList, axis=1)
#     print(RawDataDF)
# print('SET_CLASS' in RawDataDF.index.tolist())
# print(RawDataDF.loc['SET_CLASS'])

# for benchmark in benchmarks:
#     print(return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark, FixProcess=64, predict_class="D"))
#     print(return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark, FixClass="B", predict_process=256))

# "SET_CLASS"に代表される関数が、プロファイル数が2個のときだけ突飛な平均誤差率を出力する。その原因は学主データ二個だと基本的に線形モデルを採用することになるが、そうすると、実際の最適モデルが反比例や対数だった場合に予測値と実測値が大幅にずれる。したがって、プロファイル数が2個のときにだけ突飛な平均誤差率を出力する。

# 実際に予測することになる関数のみだけを、表示するようにして、出力されるデータを見やすくする(↑の話)。

# 表のタイトルをそのまま保存時のファイル名にすればええんか

# 発表資料などの資料作成に必要な素材を生成するためのノート


In [ ]:
# 問題サイズDに対する予測で、プロファイル数３の時の平均平均誤差率 < プロファイル数４の時の平均平均誤差率　となっているものを探す
Fixed="Process"
Fix=64
FixProcess=Fix
Predict="D"
predict_class=Predict
EnableTitle=False
SaveGraph=False
for benchmark in benchmarks:
    ErrorRateFixedProcess = return_ErrorRateFixedProcessAverageAndCosts(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    print(benchmark)
    ErrorRateFixedProcess

In [ ]:
for benchmark in benchmarks:
    print(f"FixProcess_{benchmark}")
    FixProcessDF = return_summarized_Fixed_dataframe(BenchMark_name = benchmark, fixed="process")
    FixProcessDFwithoutNaN = FixProcessDF.dropna(how='any', axis=0)
    FixProcessDFwithoutNaN
    indice = FixProcessDFwithoutNaN.T.index.tolist()
    columns = FixProcessDFwithoutNaN.T.columns.tolist()
    plt.figure()
    x = indice
    for column in columns:
        y = FixProcessDFwithoutNaN.loc[column]
        plt.plot(x, y, label=column)
    plt.yscale('log')
    plt.legend()
    plt.show()

In [ ]:
for benchmark in benchmarks:
    print(f"FixClass_{benchmark}")
    FixClassDF = return_summarized_Fixed_dataframe(BenchMark_name = benchmark, fixed="class")
    FixClassDFwithoutNaN = FixClassDF.dropna(how='any', axis=0)
    FixClassDFwithoutNaN
    indice = FixClassDFwithoutNaN.T.index.tolist()
    columns = FixClassDFwithoutNaN.T.columns.tolist()
    plt.figure()
    x = indice
    for column in columns:
        y = FixClassDFwithoutNaN.loc[column]
        plt.plot(x, y, label=column)
    plt.yscale('log')
    plt.legend()
    plt.show()

## 以下記述移行予定

In [ ]:
# ベンチマークCGでプロセス数を固定したもの

outputDF = 0
for benchmark in ["cg"]:
    print(f"FixProcess_{benchmark}")
    FixProcessDF = return_summarized_Fixed_dataframe(BenchMark_name = benchmark, fixed="process")
    FixProcessDFwithoutNaN = FixProcessDF.dropna(how='any', axis=0)
    outputDF = FixProcessDFwithoutNaN
    indice = FixProcessDFwithoutNaN.T.index.tolist()
    columns = FixProcessDFwithoutNaN.T.columns.tolist()
    plt.figure()
    x = indice
    for column in columns:
        y = FixProcessDFwithoutNaN.loc[column]
        plt.plot(x, y, label=column)
    plt.yscale('log')
    plt.legend()
    plt.show()
ICNVRTfixProcessDF = outputDF.loc["ICNVRT":"ICNVRT"]
ICNVRTfixProcessDF

In [ ]:
# ベンチマークCGでベンチマーククラスを固定したもの

outputDF = 0
for benchmark in ["cg"]:
    print(f"FixClass_{benchmark}")
    FixClassDF = return_summarized_Fixed_dataframe(BenchMark_name = benchmark, fixed="class")
    FixClassDFwithoutNaN = FixClassDF.dropna(how='any', axis=0)
    outputDF = FixClassDFwithoutNaN
    indice = FixClassDFwithoutNaN.T.index.tolist()
    columns = FixClassDFwithoutNaN.T.columns.tolist()
    plt.figure()
    x = indice
    for column in columns:
        y = FixClassDFwithoutNaN.loc[column]
        plt.plot(x, y, label=column)
    plt.yscale('log')
    plt.legend()
    plt.show()
ICNVRTfixClassDF = outputDF.loc["ICNVRT":"ICNVRT"]
ICNVRTfixClassDF

In [ ]:
intList = ConvertBenchmarkClasses(ICNVRTfixProcessDF.columns.tolist())
intList
ICNVRTfixProcessDF.columns = intList
ICNVRTfixProcessDF
ICNVRTfixClassDF

fixProcess用入力

```
PARAMETER p
POINTS (1) (4) (16)

REGION example
METRIC time
DATA 180347 180347 180347
DATA 1706010.0 1706010.0 1706010.0
DATA 3936140.0 3936140.0 3936140.0
```

fixProcess用出力

```
Callpath: example
	Metric: time
		Measurement point: (1.00E+00) Mean: 1.80E+05 Median: 1.80E+05
		Measurement point: (4.00E+00) Mean: 1.71E+06 Median: 1.71E+06
		Measurement point: (1.60E+01) Mean: 3.94E+06 Median: 3.94E+06
		Model: -3590464.6990329633 + 3759195.349891038 * p^(1/4)
		RSS: 8.79E+09
		Adjusted R^2: 9.98E-01
```

fixClass用入力

```
PARAMETER p
POINTS (1) (2) (4) (8) (16) (32) (64) (128)

REGION example
METRIC time
DATA 1984770.0 1984770.0 1984770.0
DATA 2263540.0 2263540.0 2263540.0
DATA 2821070.0 2821070.0 2821070.0
DATA 3936140.0 3936140.0 3936140.0
DATA 3936140.0 3936140.0 3936140.0
DATA 3936140.0 3936140.0 3936140.0
DATA 3936140.0 3936140.0 3936140.0
DATA 3936140.0 3936140.0 3936140.0
```

fixClass用出力


```
Callpath: example
	Metric: time
		Measurement point: (1.00E+00) Mean: 1.98E+06 Median: 1.98E+06
		Measurement point: (2.00E+00) Mean: 2.26E+06 Median: 2.26E+06
		Measurement point: (4.00E+00) Mean: 2.82E+06 Median: 2.82E+06
		Measurement point: (8.00E+00) Mean: 3.94E+06 Median: 3.94E+06
		Measurement point: (1.60E+01) Mean: 3.94E+06 Median: 3.94E+06
		Measurement point: (3.20E+01) Mean: 3.94E+06 Median: 3.94E+06
		Measurement point: (6.40E+01) Mean: 3.94E+06 Median: 3.94E+06
		Measurement point: (1.28E+02) Mean: 3.94E+06 Median: 3.94E+06
		Model: 2286768.3333333326 + 301997.61904761934 * log2(p)^(1)
		RSS: 2.18E+12
		Adjusted R^2: 4.96E-01
```


In [ ]:
plt.figure(figsize=(10,10),dpi=200)

# Extra-PでfixProcessデータを入力して出力したモデルの図時
plot_x = np.linspace(0.01, 256, 100)
# -3590464.6990329633 + 3759195.349891038 * p^(1/4)
plot_y = []
for x in plot_x:
    plot_y.append(-3590464.6990329633 + 3759195.349891038 * x**(1/4))
plt.plot(plot_x, plot_y, label="ExtraP")

x = [1, 4, 16]
y = [180347, 1706010, 3936140]
x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1,1)
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(x, y, "CG", "ICNVRT", test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# logモデル
model_log10 = ModelLog10(x, y, "CG", "ICNVRT", test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(x, y, "CG", "ICNVRT", test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(x, y, "CG", "ICNVRT", test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()

plt.scatter(x, y, marker="o")

In [ ]:
plt.figure(figsize=(10,10),dpi=200)

# Extra-PでfixProcessデータを入力して出力したモデルの図時
plot_x = np.linspace(0.01, 256, 100)
# -3590464.6990329633 + 3759195.349891038 * p^(1/4)
plot_y = []
for x in plot_x:
    plot_y.append(2286768.3333333326 + 301997.61904761934 * math.log2(x)**(1))
plt.plot(plot_x, plot_y, label="ExtraP")

x = [1, 2, 4, 8, 16, 32, 64, 128, 256]
y = [1984770.0, 2263540.0, 2821070.0, 3936140.0, 3936140.0, 3936140.0, 3936140.0, 3936140.0, 3936140.0]
x = np.array(x).reshape(-1,1)
y = np.array(y).reshape(-1, 1)
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(x, y, "CG", "ICNVRT", test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(x, y, "CG", "ICNVRT", test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(x, y, "CG", "ICNVRT", test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(x, y, "CG", "ICNVRT", test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()

plt.scatter(x, y, marker="o")